In [ ]:
#CELL 1
#@title Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the audio player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# @title # **Cell 1 - Installation w/ Model Downloading** { display-mode: "form" }
# @markdown ---
# @markdown # Download Model
# Select model and branch
repo_id = "royallab/airoboros-mistral2.2-7b-exl2" # @param {type:"string"}
revision = "6bpw" # @param {type:"string"}
# @markdown ---

# Install tabbyAPI
%cd /content/

!git clone https://github.com/theroyallab/tabbyAPI
%cd tabbyAPI
!pip install https://github.com/turboderp/exllamav2/releases/download/v0.0.10/exllamav2-0.0.10+cu118-cp310-cp310-linux_x86_64.whl -q
!pip install -r requirements.txt -q
!pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.3.6/flash_attn-2.3.6+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl -q
!pip install huggingface-hub -q

# Download cloudflared tunnel
%cd /content/tabbyAPI/

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod a+x cloudflared

# Download model repo
%cd /content/tabbyAPI/

from huggingface_hub import snapshot_download
snapshot_download(repo_id=repo_id, revision=revision, local_dir=f"./models/{repo_id.replace('/', '_')}")

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.0-1_all.deb
!dpkg -i cuda-keyring_1.0-1_all.deb
!apt-get -qq update
!apt-get -y -qq install cuda

print(f"Model dir: './models/{repo_id.replace('/', '_')}'")

In [ ]:
# @title # **Cell 2 - Edit Config and Start Tabby** { display-mode: "form" }
# @markdown ---
# @markdown # Edit Config

model = repo_id.replace('/', '_')
ContextSize = 4096 # @param {type:"raw"}
RopeScale = 1.0 # @param {type:"raw"}
RopeAlpha = 1.0 # @param {type:"raw"}

# Setup Config - edit parameters to fit your needs
%cd /content/tabbyAPI/

write = f'''
# Sample YAML file for configuration.
# Comment out values as needed. Every value has a default within the application.

# Unless specified in the comments, DO NOT put these options in quotes!
# You can use https://www.yamllint.com/ if you want to check your YAML formatting.

# Options for networking
network:
  # The IP to host on (default: 127.0.0.1).
  # Use 0.0.0.0 to expose on all network adapters
  host: 0.0.0.0

  # The port to host on (default: 5000)
  port: 5000

# Options for model overrides and loading
model:
  # Overrides the directory to look for models (default: models)
  # Windows users, DO NOT put this path in quotes! This directory will be invalid otherwise.
  model_dir: models

  # An initial model to load. Make sure the model is located in the model directory!
  # A model can be loaded later via the API.
  model_name: {model}

  # Sends dummy model names when the models endpoint is queried
  # Enable this if the program is looking for a specific OAI model
  use_dummy_models: False

  # The below parameters apply only if model_name is set

  # Maximum model context length (default: 4096)
  max_seq_len: {ContextSize}

  # Automatically allocate resources to GPUs (default: True)
  gpu_split_auto: True

  # An integer array of GBs of vram to split between GPUs (default: [])
  # gpu_split: [20.6, 24]

  # Rope scaling parameters (default: 1.0)
  rope_scale: {RopeScale}
  rope_alpha: {RopeAlpha}

  # Disable Flash-attention 2. Set to True for GPUs lower than Nvidia's 3000 series. (default: False)
  no_flash_attention: False

  # Enable 8 bit cache mode for VRAM savings (slight performance hit). Possible values FP16, FP8. (default: FP16)
  cache_mode: FP16

  # Options for draft models (speculative decoding). This will use more VRAM!
  draft:
    # Overrides the directory to look for draft (default: models)
    # draft_model_dir: Your draft model directory path

    # An initial draft model to load. Make sure this model is located in the model directory!
    # A draft model can be loaded later via the API.
    # draft_model_name: A model name

    # Rope parameters for draft models (default: 1.0)
    # draft_rope_alpha: 1.0
'''
with open("./config.yml", "w") as file:
    file.write(write)
!cat config.yml

%cd /content/tabbyAPI/

!python main.py & ./cloudflared tunnel --url localhost:5000